In [ ]:
# --------------------------------------------------------------------------------------
# Cell 1
# --------------------------------------------------------------------------------------
import boto3
import requests
import uuid
import json


client = boto3.client('rekognition', region_name='us-east-1')

BUCKET='aws-analytics-course'

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 2
# --------------------------------------------------------------------------------------

photo='raw/images/dog.jpg'

response = client.detect_labels(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}}, MaxLabels=10)

for label in response['Labels']:
    print ("Label: " + label['Name'])
    print ("Confidence: " + str(label['Confidence']))

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 3
# --------------------------------------------------------------------------------------
photo='raw/images/police-sign.jpg'

response = client.detect_labels(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}}, MaxLabels=10)

for label in response['Labels']:
    print ("Label: " + label['Name'])
    print ("Confidence: " + str(label['Confidence']))

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 4
# --------------------------------------------------------------------------------------
photo='raw/images/police-sign.jpg'

response = client.detect_text(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}})

textDetections=response['TextDetections']
print ('Detected text')
for text in textDetections:
    print ('Detected text:' + text['DetectedText'])
    print ('Confidence: ' + "{:.2f}".format(text['Confidence']) + "%")
    print ('Id: {}'.format(text['Id']))
    if 'ParentId' in text:
        print ('Parent Id: {}'.format(text['ParentId']))
    print ('Type:' + text['Type'])
    print

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 5
# --------------------------------------------------------------------------------------
response = requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Bill_Gates_2017_%28cropped%29.jpg/330px-Bill_Gates_2017_%28cropped%29.jpg')
response_content = response.content

rekognition_response = client.recognize_celebrities(Image={'Bytes': response_content})

print(rekognition_response)

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 6
# --------------------------------------------------------------------------------------
KEY_SOURCE = "raw/images/tiger1.jpg"
KEY_TARGET = "raw/images/tiger2.jpg"

def compare_faces(bucket, key, bucket_target, key_target, threshold=80, region="us-east-1"):
    rekognition = boto3.client("rekognition", region)
    response = client.compare_faces(
    SourceImage={
          "S3Object": {
          "Bucket": bucket,
          "Name": key,
          }
        },
        TargetImage={
           "S3Object": {
           "Bucket": bucket_target,
           "Name": key_target,
           }
        },
        SimilarityThreshold=threshold,
        )
    return response['SourceImageFace'], response['FaceMatches']


source_face, matches = compare_faces(BUCKET, KEY_SOURCE, BUCKET, KEY_TARGET)

# the main source face
#print ("Source Face ({Confidence}%)"+format(**source_face))
#print ("matches: "+str(matches[0]))

# one match for each target face
for match in matches[0]:
    #print(matches[0]['Face']['Confidence'])
    print ("Target Face ({Confidence}%)"+str(matches[0]['Face']['Confidence']))
    print ("  Similarity : {}%"+str(matches[0]['Similarity']))



In [ ]:
# --------------------------------------------------------------------------------------
# Cell 7
# --------------------------------------------------------------------------------------
KEY_SOURCE = "raw/images/tiger1.jpg"
KEY_TARGET = "raw/images/tiger_not.jpg"

def compare_faces(bucket, key, bucket_target, key_target, threshold=80, region="eu-west-1"):
    rekognition = boto3.client("rekognition", region)
    response = client.compare_faces(
    SourceImage={
          "S3Object": {
          "Bucket": bucket,
          "Name": key,
          }
        },
        TargetImage={
           "S3Object": {
           "Bucket": bucket_target,
           "Name": key_target,
           }
        },
        SimilarityThreshold=threshold,
        )
    return response['SourceImageFace'], response['FaceMatches']


source_face, matches = compare_faces(BUCKET, KEY_SOURCE, BUCKET, KEY_TARGET)

# the main source face
#print "Source Face ({Confidence}%)".format(**source_face)
if not matches:
    print("Not a Match")



In [ ]:
# --------------------------------------------------------------------------------------
# Cell 8
# --------------------------------------------------------------------------------------
KEY = "raw/images/tiger1.jpg"
IMAGE_ID = uuid.uuid4().hex  

COLLECTION = "rek-collection-1"
client.create_collection(CollectionId=COLLECTION)
client.list_collections()

def index_faces(bucket, key, collection_id, image_id=None, attributes=(), region="us-east-1"):

    response = client.index_faces(
                                 Image={
                                 "S3Object": {
                                 "Bucket": bucket,
                                 "Name": key,
                                 }
                                 },
                                 CollectionId=collection_id,
                                 ExternalImageId=image_id,
                                     DetectionAttributes=attributes,
                                 )
    return response['FaceRecords']


for record in index_faces(BUCKET, KEY, COLLECTION, IMAGE_ID):
    face = record['Face']
    print ("Face ({}%)".format(face['Confidence']))
    print ("FaceId: {}".format(face['FaceId']))
    print ("ImageId: {}".format(face['ImageId']))


In [ ]:
# --------------------------------------------------------------------------------------
# Cell 9
# --------------------------------------------------------------------------------------
KEY = "raw/images/search_tiger.jpg"

def search_faces_by_image(bucket, key, collection_id, threshold=80, region="eu-west-1"):
    response = client.search_faces_by_image(
                         Image={
                         "S3Object": {
                         "Bucket": bucket,
                         "Name": key,
                         }
                         },
                         CollectionId=collection_id,
                         FaceMatchThreshold=threshold,
                         )
    return response['FaceMatches']

for record in search_faces_by_image(BUCKET, KEY, COLLECTION):
        face = record['Face']
        print ("Matched Face ({}%)".format(record['Similarity']))
        print ("  FaceId : {}".format(face['FaceId']))
        print ("  ImageId : {}".format(face['ExternalImageId']))

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 10
# --------------------------------------------------------------------------------------
KEY = "raw/images/will.jpg"

def search_faces_by_image(bucket, key, collection_id, threshold=80, region="eu-west-1"):
    response = client.search_faces_by_image(
                 Image={
                      "S3Object": {
                      "Bucket": bucket,
                      "Name": key,
                 }
                 },
                 CollectionId=collection_id,
                 FaceMatchThreshold=threshold,
                 )
    return response['FaceMatches']

for record in search_faces_by_image(BUCKET, KEY, COLLECTION):
        face = record['Face']
        print ("Matched Face ({}%)".format(record['Similarity']))
        print ("  FaceId : {}".format(face['FaceId']))
        print ("  ImageId : {}".format(face['ExternalImageId']))
else:
    print('Image not in our database')

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 11
# --------------------------------------------------------------------------------------
client.delete_collection(CollectionId=COLLECTION)
client.list_collections()

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 12
# --------------------------------------------------------------------------------------
photo='raw/images/pistol.jpg'

response = client.detect_labels(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}}, MaxLabels=10)

for label in response['Labels']:
    print ("Label: " + label['Name'])
    print ("Confidence: " + str(label['Confidence']))

In [ ]:
# --------------------------------------------------------------------------------------
# Cell 13
# --------------------------------------------------------------------------------------
photo='raw/images/angry_wife.jpg'

response = client.detect_faces(Image={'S3Object':{'Bucket':BUCKET,'Name':photo}}, Attributes=['ALL'])
 
for face in response['FaceDetails']:
    for emotion in face['Emotions']:
        #print "  {Type} : {Confidence}%".format(**emotion)
        if (emotion['Type'] == 'ANGRY' and emotion['Confidence'] > 90):
           print ("It late, time to go home, but wait your forgot your wedding anniversary today. Your wife is ready to welcome you!! ")